<a href="https://colab.research.google.com/github/redwon007/JobShopPSO/blob/main/15x15_jobShop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
from copy import deepcopy
import plotly.figure_factory as ff
import plotly.express as px
import pandas as pd
from datetime import datetime
import math
from random import randint, random

In [ ]:
nbMachines = input("nbMachines?")
nbJobs = input("nbJobs?")
nbDataSet = int(input("nbDataSet?"))

path = "tai" + nbMachines + "_" + nbJobs + ".txt"
fichier = open(path, "r")

for i in range(34*(nbDataSet-1) + 3 ):
    line = fichier.readline()
    
times = []
for i in range(15):
    line = [int(i) for i in fichier.readline().strip().split()]
    times.append(line)
    
line = fichier.readline()#read line "Machines\n"

machines = []
for i in range(15):
    line = [int(i) for i in fichier.readline().strip().split()]
    machines.append(line)
    
fichier.close()

nbMachines?15
nbJobs?15
nbDataSet?1


In [ ]:
print(times)
print(machines)

[[94, 66, 10, 53, 26, 15, 65, 82, 10, 27, 93, 92, 96, 70, 83], [74, 31, 88, 51, 57, 78, 8, 7, 91, 79, 18, 51, 18, 99, 33], [4, 82, 40, 86, 50, 54, 21, 6, 54, 68, 82, 20, 39, 35, 68], [73, 23, 30, 30, 53, 94, 58, 93, 32, 91, 30, 56, 27, 92, 9], [78, 23, 21, 60, 36, 29, 95, 99, 79, 76, 93, 42, 52, 42, 96], [29, 61, 88, 70, 16, 31, 65, 83, 78, 26, 50, 87, 62, 14, 30], [18, 75, 20, 4, 91, 68, 19, 54, 85, 73, 43, 24, 37, 87, 66], [32, 52, 9, 49, 61, 35, 99, 62, 6, 62, 7, 80, 3, 57, 7], [85, 30, 96, 91, 13, 87, 82, 83, 78, 56, 85, 8, 66, 88, 15], [5, 59, 30, 60, 41, 17, 66, 89, 78, 88, 69, 45, 82, 6, 13], [90, 27, 1, 8, 91, 80, 89, 49, 32, 28, 90, 93, 6, 35, 73], [47, 43, 75, 8, 51, 3, 84, 34, 28, 60, 69, 45, 67, 58, 87], [65, 62, 97, 20, 31, 33, 33, 77, 50, 80, 48, 90, 75, 96, 44], [28, 21, 51, 75, 17, 89, 59, 56, 63, 18, 17, 30, 16, 7, 35], [57, 16, 42, 34, 37, 26, 68, 73, 5, 8, 12, 87, 83, 20, 97]]
[[7, 13, 5, 8, 4, 3, 11, 12, 9, 15, 10, 14, 6, 1, 2], [5, 6, 8, 15, 14, 9, 12, 10, 7, 11, 1

In [ ]:
def vectorSolToMatrixSol(vectorSol):
    matrixSol = [[] for i in range(len(machines))]
    operation = [1] * len(machines[0])
    for job in vectorSol:
        job = int(job)
        matrixSol[machines[job-1][operation[job-1]-1]-1].append((job,operation[job-1]))
        operation[job-1] += 1
    return(matrixSol)


In [ ]:
def createGantt(solution):
    gantt = deepcopy(solution)
    nbJobs = len(machines[0])
    toPlaceList = [1] * nbJobs
    endTimeList = [0] * nbJobs
    i,j = 0,0
    while(i<len(machines)):
    #for i in range(len(machines)):
        
        while(j<nbJobs):
        #for j in range(nbJobs):
            #print(i,j, " ", toPlaceList , " " , endTimeList)
            refresh = False
            for job, toPlace in enumerate(toPlaceList):
                
                if(solution[i][j][0] == job+1 and solution[i][j][1] == toPlace):
                    if(j==0 or gantt[i][j-1] != solution[i][j-1]):
                        #print(toPlaceList , " " , endTimeList)      
                        #print(i,j)
                        refresh = True
                        if(j == 0):
                            previous = 0
                        else:
                            previous = gantt[i][j-1][1]
                        startTime = max(previous, endTimeList[job])
                        endTime = startTime + times[job][solution[i][j][1]-1]
                        gantt[i][j] = (startTime, endTime)
                        toPlaceList[job] += 1
                        endTimeList[job] = endTime
                        #j = 0
            if(refresh == True):
                break
            j += 1
        i += 1
        j = 0    
        if(refresh == True):
            i=0
    return(gantt)
    

In [ ]:
def makespan(vectorSolution):
    gantt = createGantt(vectorSolToMatrixSol(vectorSolution))
    makespan = max(gantt[0][-1])
    for lastOp in gantt:     
      lastRelease = max(lastOp[-1])
      if(lastRelease > makespan):
        makespan = lastRelease
    return(makespan)

In [ ]:
def computeNextVelocity(velocity, pos, bPos, bbPos):
    nextVelocity = velocity + (bPos - pos) + (bbPos - pos)
    return nextVelocity

In [ ]:
def arrangeSol(sol, continuousSol):
    newSol = np.array([])
    while(np.size(sol) != 0):
        minIndex = np.where(continuousSol == np.amin(continuousSol))[0]
        
        for i in minIndex:
            newSol = np.append(newSol, sol[i])
        inds_to_delete = sorted(minIndex, reverse=True)
        for i in inds_to_delete:
            sol = np.delete(sol, i)
            continuousSol = np.delete(continuousSol, i)
    return(newSol)

In [ ]:
def firstNotSchedule(newPosition, position):
  scheduled = [0] * len(machines)
  for i, job in enumerate(position):
    if(np.count_nonzero(newPosition == job) == scheduled[int(job)-1]):
      return(job)
    else:
      scheduled[int(job)-1] +=1

In [ ]:
def newParticle(particle, pBestParticle, gBestParticle):
  newPosition = np.array([])
  #pIndex, pBIndex, gBIndex = 0, 0, 0
  for j in range(len(particle)):
    i = 0
    nbJobs = len(times)
    wc = randint(0, 10)
    wb = randint(0, 10-wc)
    #wg = 10-wc-wb
    #wc /= 10
    #wb/= 10
    #wg /= 10
    wc, wb, wg = 0.2, 0.3, 0.5
    u = random()
    if(u <= wc):
      newPosition = np.append(newPosition, firstNotSchedule(newPosition, particle))
    elif(u <= wb + wg):
      newPosition = np.append(newPosition, firstNotSchedule(newPosition, pBestParticle))
    elif(wc+wb < u):
      newPosition = np.append(newPosition, firstNotSchedule(newPosition, gBestParticle))

  return(newPosition)


In [ ]:
def createSol(nbJobs, nbMachines):
  sol = np.array([])
  for i in range(nbMachines):
    for j in range(1, nbJobs + 1):
      sol = np.append(sol, j)
  return(sol)

In [ ]:
sol = createSol(15, 15)
print(sol)

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.  1.  2.  3.
  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.  1.  2.  3.  4.  5.  6.
  7.  8.  9. 10. 11. 12. 13. 14. 15.  1.  2.  3.  4.  5.  6.  7.  8.  9.
 10. 11. 12. 13. 14. 15.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.
 13. 14. 15.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.
  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.  1.  2.  3.
  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.  1.  2.  3.  4.  5.  6.
  7.  8.  9. 10. 11. 12. 13. 14. 15.  1.  2.  3.  4.  5.  6.  7.  8.  9.
 10. 11. 12. 13. 14. 15.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.
 13. 14. 15.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.
  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.  1.  2.  3.
  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.  1.  2.  3.  4.  5.  6.
  7.  8.  9. 10. 11. 12. 13. 14. 15.]


In [ ]:
def jobShopPSO():
    swarm = [{"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
        ]
    gBestParticle = swarm[0]["position"]
    gBestParticleCmax = makespan(gBestParticle)
    for i, particle in enumerate(swarm):
        particle["pBestParticle"] = particle["position"]
        particle["pBestParticleCmax"] = makespan(particle["pBestParticle"])
        particle["velocity"] = 0
        if(particle["pBestParticleCmax"] < gBestParticleCmax):
            gBestParticle = particle["position"]
            gBestParticleCmax = particle["pBestParticleCmax"]
        
    iteration = 0
    while(iteration < 200):
        iteration +=1
        print('\n\niteration', iteration)
        for i, particle in enumerate(swarm):
            #particle["velocity"] = computeNextVelocity(particle["velocity"], particle["position"], particle["pBestParticle"], gBestParticle)
            particle["position"] = newParticle(particle["position"], particle["pBestParticle"], gBestParticle)
            pCmax = makespan(particle["position"])
            if(pCmax < particle["pBestParticleCmax"]):
                particle["pBestParticle"] = particle["position"]
                particle["pBestParticleCmax"] = pCmax
                if(pCmax < gBestParticleCmax):
                    gBestParticle = particle["position"]
                    gBestParticleCmax = pCmax
            print("particle",i," cmax = ", pCmax)
    return((gBestParticle, gBestParticleCmax))

In [ ]:
a = jobShopPSO()

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
particle 20  cmax =  2224
particle 21  cmax =  1676
particle 22  cmax =  1879
particle 23  cmax =  1795
particle 24  cmax =  1598
particle 25  cmax =  1882
particle 26  cmax =  1683
particle 27  cmax =  1867
particle 28  cmax =  1598
particle 29  cmax =  1629
particle 30  cmax =  1744
particle 31  cmax =  1790
particle 32  cmax =  1741
particle 33  cmax =  1549
particle 34  cmax =  1779
particle 35  cmax =  1964
particle 36  cmax =  1642
particle 37  cmax =  1577
particle 38  cmax =  1642
particle 39  cmax =  1823
particle 40  cmax =  1995
particle 41  cmax =  1624
particle 42  cmax =  1943
particle 43  cmax =  1727
particle 44  cmax =  1816
particle 45  cmax =  1805
particle 46  cmax =  1582
particle 47  cmax =  1521
particle 48  cmax =  1928
particle 49  cmax =  1659
particle 50  cmax =  1648


iteration 109
particle 0  cmax =  1712
particle 1  cmax =  2053
particle 2  cmax =  1806
particle 3  cmax =  1929


In [ ]:
print(a)

(array([13., 14.,  1.,  6.,  4.,  2.,  7.,  5.,  6.,  3., 12., 11., 15.,
        3., 10.,  1.,  8.,  9.,  1.,  2.,  9.,  4., 12., 11., 14.,  7.,
       13., 12.,  2.,  4.,  8., 14.,  3., 10., 15.,  9.,  5.,  8.,  9.,
       11., 12., 14., 15., 13., 15., 14.,  5., 12.,  1.,  7.,  4.,  2.,
        7.,  5., 10., 13., 14.,  1.,  6., 11., 13.,  6., 10.,  9., 11.,
        3.,  3., 10.,  8., 12., 15.,  6., 10.,  2.,  5.,  8., 11., 10.,
       12.,  6.,  5.,  9., 13.,  7.,  4.,  2.,  8., 12., 11.,  6.,  1.,
       12.,  3., 15.,  6., 13.,  8., 14.,  5.,  1., 11., 15., 10.,  2.,
        4., 13.,  4.,  3., 13.,  8.,  9.,  4.,  6.,  7., 10., 15.,  1.,
        9., 10.,  6.,  1.,  3.,  2., 14., 12.,  5.,  9., 15.,  5.,  2.,
       13.,  7., 15.,  4., 14.,  7., 11.,  8., 11., 14.,  3.,  9.,  2.,
        4.,  1.,  7.,  3.,  9.,  1., 10.,  6., 15.,  4., 12., 14.,  4.,
       13.,  5., 12., 13.,  3., 10.,  8.,  5.,  2.,  7.,  9., 11., 11.,
       10.,  2.,  2., 13.,  8., 15.,  6., 12.,  3.,  3.,  1., 1

In [ ]:
print(vectorSolToMatrixSol(a[0]))
print(a[1])
createGantt(vectorSolToMatrixSol(a[0]))

[[(12, 3), (8, 3), (11, 4), (9, 5), (10, 6), (4, 6), (13, 8), (15, 10), (3, 9), (2, 11), (6, 12), (7, 13), (1, 14), (5, 14), (14, 15)], [(3, 1), (9, 6), (15, 6), (8, 8), (6, 10), (14, 8), (7, 7), (10, 11), (13, 11), (12, 12), (5, 11), (11, 12), (2, 14), (4, 14), (1, 15)], [(12, 1), (4, 2), (13, 4), (10, 4), (1, 6), (5, 7), (9, 7), (6, 9), (8, 9), (3, 10), (7, 11), (15, 13), (14, 14), (2, 15), (11, 15)], [(6, 2), (7, 2), (1, 5), (13, 5), (10, 8), (11, 11), (2, 12), (15, 12), (12, 13), (8, 12), (14, 13), (3, 14), (9, 15), (5, 15), (4, 15)], [(2, 1), (11, 1), (1, 3), (14, 3), (15, 5), (5, 5), (6, 6), (4, 8), (13, 10), (9, 10), (7, 9), (8, 11), (3, 13), (10, 14), (12, 15)], [(13, 1), (6, 1), (4, 1), (2, 2), (8, 2), (14, 5), (11, 5), (12, 8), (9, 8), (3, 8), (5, 10), (7, 10), (10, 15), (1, 13), (15, 15)], [(1, 1), (10, 1), (9, 3), (15, 4), (5, 3), (12, 5), (4, 4), (14, 6), (3, 5), (13, 6), (11, 7), (7, 6), (2, 9), (6, 14), (8, 14)], [(5, 1), (11, 2), (2, 3), (1, 4), (7, 3), (8, 4), (12, 7),

[[(90, 165),
  (250, 259),
  (259, 267),
  (359, 372),
  (404, 421),
  (457, 551),
  (609, 686),
  (745, 753),
  (753, 807),
  (978, 996),
  (1028, 1115),
  (1167, 1204),
  (1287, 1357),
  (1357, 1399),
  (1399, 1434)],
 [(0, 4),
  (372, 459),
  (459, 485),
  (667, 729),
  (729, 755),
  (755, 811),
  (811, 830),
  (830, 899),
  (899, 947),
  (947, 992),
  (994, 1087),
  (1087, 1180),
  (1180, 1279),
  (1279, 1371),
  (1371, 1454)],
 [(0, 47),
  (167, 190),
  (269, 289),
  (290, 350),
  (365, 380),
  (448, 543),
  (543, 625),
  (649, 727),
  (729, 735),
  (807, 875),
  (1081, 1124),
  (1134, 1217),
  (1297, 1304),
  (1304, 1337),
  (1401, 1474)],
 [(94, 155),
  (155, 230),
  (339, 365),
  (365, 396),
  (533, 622),
  (899, 989),
  (996, 1047),
  (1047, 1134),
  (1134, 1201),
  (1201, 1281),
  (1281, 1297),
  (1297, 1332),
  (1332, 1347),
  (1399, 1495),
  (1495, 1504)],
 [(0, 74),
  (74, 164),
  (164, 174),
  (174, 225),
  (302, 339),
  (383, 419),
  (419, 450),
  (694, 787),
  (787, 867

In [ ]:

df = pd.DataFrame([
    dict(Task="Job A", Start=convert_to_datetime(0), Finish=convert_to_datetime(3), Resource="Alex"),
    dict(Task="Job B", Start=convert_to_datetime(4), Finish=convert_to_datetime(5), Resource="Alex"),
    dict(Task="Job C", Start=convert_to_datetime(0), Finish=convert_to_datetime(2), Resource="Max")
])
#Maurice clerc PSO

NameError: name 'convert_to_datetime' is not defined

In [ ]:
def convert_to_datetime(x):
  return datetime.fromtimestamp(31536000+x*24*3600).strftime("%Y-%d-%m")

In [ ]:
num_tick_labels = np.linspace(start = 0, stop = 10, num = 11, dtype = int)
date_ticks = [convert_to_datetime(x) for x in num_tick_labels]

fig = ff.create_gantt(df)
fig.layout.xaxis.update({
        'tickvals' : date_ticks,
        'ticktext' : num_tick_labels
        })
fig.write_html('first_figure.html', auto_open=True)
fig.show()